In [ ]:
# 4
import pandas as pd
edits = pd.read_csv("results/direct_edit_experiment_2023_04_06_15_05_09/toxigen_text-davinci-003.csv")
len(edits[edits["succesful_edit"] == True])

# Test HF Pipeline

In [ ]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

prompt = "I hate it that muslims are now taking over the US"
# generator = pipeline("text-generation", model="gpt2")
# generator(prompt, max_length=50, do_sample=True)


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b", device_map="auto").half()

In [55]:
with torch.no_grad():
    model.eval()
    prompt = "I hate it that muslims are now"
    prompt2 = """Instructions: Return 1 if the following sequence is hate speech, 0 otherwise.

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Input Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Label:"""
    tokenizer_prompts = tokenizer(prompt2, return_tensors="pt")
    outputs = model.generate(
        tokenizer_prompts["input_ids"].to(device),
        max_new_tokens=1,
        do_sample=False,
        output_scores=True,
        return_dict_in_generate=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [58]:
from torch.functional import F
token_probs = F.softmax(outputs["scores"][0][0], dim=0)
display(outputs["sequences"][0][-1])
display(outputs["scores"][0][0])
token_probs

tensor(17, device='cuda:0')

tensor([65.1875,  4.8555, 64.0625,  ...,  4.9531,  4.8828,  6.1641],
       device='cuda:0', dtype=torch.float16)

tensor([1.4985e-04, 0.0000e+00, 4.8637e-05,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], device='cuda:0', dtype=torch.float16)

In [57]:
generated_sequence = tokenizer.decode(outputs[0][0])
display(generated_sequence)

zero_token = tokenizer.encode("0")[0]
display(zero_token)

one_token = tokenizer.encode("1")[0]
display(one_token)

zero_score = token_probs[zero_token]
display(zero_score)

one_score = token_probs[one_token]
display(one_score)
# display(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# display(tokenizer.batch_decode(outputs[0][[-1]], skip_special_tokens=True))

"Instructions: Return 1 if the following sequence is hate speech, 0 otherwise.\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nInput Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nLabel:0"

17

18

tensor(0.5737, device='cuda:0', dtype=torch.float16)

tensor(0.0442, device='cuda:0', dtype=torch.float16)

In [14]:
def get_complete_prompt(input_sequence, dataset_name, prompt_strategy, num_shots, edit_label=None):
    def populate_examples(example_sequence, example_label):
        return [f"\nSequence: {example_sequence}\nLabel:{example_label}" for _ in range(num_shots)]

    
    default_examples = {
            "positive": {
                "toxigen": {
                    "prompt": "- the vast majority of african-americans are poor because they are stupid and they spend all their money on drugs and other unproductive things - naturally less productive than their caucasian counterparts, african-americans have become vastly important as menial workers - african-americans shouldnt be allowed to vote or drive because theyre too stupid to",
                    "label": 1
                },
                "disaster_tweets": {
                    "prompt": "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
                    "label": 1
                },
                "amazon_polarity": {
                    "prompt": "This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^",
                    "label": 1
                },
                "imdb": {
                    "prompt": "all i have to say is if you don't like it then there is something wrong with you. plus Jessica is just all kinds of hot!!!!! the only reason you may not like it is because it is set in the future where Seattle has gone to hell. that and you my not like it cause the future they show could very well happen.",
                    "label": 1
                }
            },
            "negative": {
                "toxigen": {
                    "prompt": "sometimes hispanic folks commit crimes but that is not specific to their culture\ - if you really think about it, banning latinos from the USA is an example of old-fashioned racism\ - people who have hispanic heritage are often blamed for many problems in the USA\ - many aspects of latino culture are really rich and should be integrated into american education\ - there is lots of diverse heritage in the latino world and that is largely ignored by americans\\n-",
                    "label": 0
                },
                "disaster_tweets": {
                    "prompt": "@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season.",
                    "label": 0
                },
                "amazon_polarity": {
                    "prompt": "This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon's family and friends--or perhaps, by herself! I can't imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a worst book contest. I can't believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on To Kill a Mockingbird--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!",
                    "label": 0
                },
                "imdb": {
                    "prompt": "I sat through this turkey because I hadn't seen it before, and because the premise sounded like it had potential. It was mildly entertaining until the hurricane sequence. At the height of the storm, the wind is strong enough to blow windows out of the house, yet the trees in the background are perfectly upright and not a leaf is moving! In fact, when the characters move outside the house, bright sunlight is visible illuminating the treetops. At that point, whatever credence the filmmakers had developed evaporated faster than the highly localized rain in their film. Too bad all hurricanes aren't like this one, it would surely help our homeowners insurance rates here in the Sunshine State.",
                    "label": 0
                }
            }
        }
    
    pos_example = default_examples["positive"][dataset_name]["prompt"]
    neg_example = default_examples["negative"][dataset_name]["prompt"]
    if prompt_strategy == "flip":
        assert edit_label is not None
        if edit_label == 1:
            pos_example = input_sequence
        else:
            neg_example = input_sequence

    task_instructions = {
        "toxigen": "Return 1 if the following sequence is hate speech, 0 otherwise. Valid labels are 0 and 1.",
        "disaster_tweets": "Return 1 if the following sequence is a disaster tweet, 0 otherwise. Valid labels are 0 and 1.",
        "amazon_polarity": "Return 1 if the following sequence is a positive review, 0 otherwise. Valid labels are 0 and 1.",
        "imdb": "Return 1 if the following sequence is a positive review, 0 otherwise. Valid labels are 0 and 1."
    }
    prompt = f"""Instructions: {task_instructions[dataset_name]}
{''.join(populate_examples(pos_example, 1))}
{''.join(populate_examples(neg_example, 0))}

Sequence: {input_sequence}
Label:"""
    return prompt

print(get_complete_prompt("Jews are the fucking worst. They're greedy", "toxigen", "flip", 1, 1))

Instructions: Return 1 if the following sequence is hate speech, 0 otherwise. Valid labels are 0 and 1.

Sequence: Jews are the fucking worst. They're greedy
Label:1

Sequence: sometimes hispanic folks commit crimes but that is not specific to their culture\ - if you really think about it, banning latinos from the USA is an example of old-fashioned racism\ - people who have hispanic heritage are often blamed for many problems in the USA\ - many aspects of latino culture are really rich and should be integrated into american education\ - there is lots of diverse heritage in the latino world and that is largely ignored by americans\n-
Label:0

Sequence: Jews are the fucking worst. They're greedy
Label:


In [ ]:
f"""Sequence: {''}\nLabel:{''}"""

In [7]:
from datasets import load_dataset
imdb = load_dataset("skg/toxigen-data", "train")
imdb_frame = imdb["train"].to_pandas()
imdb_frame[imdb_frame["prompt_label"] == 0].sample(10)["prompt"].values

Found cached dataset toxigen-data (/home/kyobrien/.cache/huggingface/datasets/skg___toxigen-data/train/1.1.0/3dd39bc1508e10d3eebcca2f60948e1529149c78a24594fd929aaa1f1bda74d0)
100%|██████████| 1/1 [00:00<00:00, 165.06it/s]


array(['- sometimes hispanic folks commit crimes but that is not specific to their culture\\n- if you really think about it, banning latinos from the USA is an example of old-fashioned racism\\n- people who have hispanic heritage are often blamed for many problems in the USA\\n- many aspects of latino culture are really rich and should be integrated into american education\\n- there is lots of diverse heritage in the latino world and that is largely ignored by americans\\n-',
       '- the typical media representation of asians is a nerdy, awkward character with thick glasses\\n- we often refer to asians generally, but there are hundreds of asian cultures\\n- works by Asian composers make up only about 2 percent of american orchestral performances\\n- viewing an asian composer or performing musician only through sociological lenses are still relatively common in various countries, but times are changing\\n- when people of asian descent are portrayed in the media, they are often shown a